# Hermitcraft Season 9 Stats

This notebook summarize minecraft stats files across all worlds saved on the local machine (currently OS X).  I would love to see this ported to work on Windows files, but additional work is required.

The emphasis is on creating plots that span worlds and compare worlds. Increasingly the focus is on statistics specific to hardcore mode.

Successfully executing the code will create a html file:  MinecraftStats.html.  The local_user variable needs to be modified to work your system. Comments in the notebook itself address the data analysis process.


In [10]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
import gzip
import hvplot.pandas
from bokeh.resources import INLINE
import bokeh
import panel as pn
import holoviews as hv
import os

pn.extension(sizing_mode="stretch_width")
hv.extension("bokeh")

hermits = [
    {
        "profile": "Docm77",
        "hermit_key": "05e88dce-714d-4218-be77-fade8b5dfa3c",
    },
    {
        "profile": "Xisuma",
        "hermit_key": "21ef397c-3a76-4eb7-aa17-a99d3fc658e2",
    },
    {
        "profile": "Renthedog",
        "hermit_key": "2dd0cc3b-0825-4c3e-bd99-3bf07ef27447",
    },
    {
        "profile": "VintageBeef",
        "hermit_key": "2f723150-24de-44ff-aeee-87c75f7c7a9e",
    },
    {
        "profile": "iJevin",
        "hermit_key": "3f28c559-0898-4be1-9f20-9fd37ca9cd22",
    },
    {
        "profile": "joehillssays",
        "hermit_key": "53bae456-dbbb-4c2f-8c79-9e8ec26c8382",
    },
    {
        "profile": "GeminiTay",
        "hermit_key": "5a1839d2-cecc-4c85-aa08-b346f9f772a1",
    },
    {
        "profile": "grian",
        "hermit_key": "5f8eb73b-25be-4c5a-a50f-d27d65e30ca0",
    },
    {
        "profile": "Tango",
        "hermit_key": "62fec5a3-1896-4beb-94e0-36e34898c787",
    },
    {
        "profile": "BdoubleO100",
        "hermit_key": "7163fbce-39ac-4a02-b836-a991c45d2dd1",
    },
    {
        "profile": "PearlescentMoon",
        "hermit_key": "75c863ae-bb92-486d-911c-53030c552be0",
    },
    {
        "profile": "iskall85",
        "hermit_key": "7ed3587b-e656-4689-90d6-08e11daaf907",
    },
    {
        "profile": "xBCrafted",
        "hermit_key": "826cdcff-ccb0-42c5-9104-fcd4bb4e7f73",
    },
    {
        "profile": "falsesymmetry",
        "hermit_key": "87d91548-6f18-491f-a267-7833caa5d7d8",
    },
    {
        "profile": "cubfan135",
        "hermit_key": "88e2afec-6f2e-4a34-a96a-de61730bd3ca",
    },
    {
        "profile": "Welsknight",
        "hermit_key": "8fc22d29-4bac-4abe-84d4-7920ed4afe47",
    },
    {
        "profile": "Etho",
        "hermit_key": "93b459be-ce4f-4700-b457-c1aa91b3b687",
    },
    {
        "profile": "ZombieCleo",
        "hermit_key": "a3075fa7-ec13-49a2-aa47-6529e8b7daf2",
    },
    {
        "profile": "Mumbo",
        "hermit_key": "ac224782-efff-4296-b08c-dbde8e47abdb",
    },
    {
        "profile": "hypnotizd",
        "hermit_key": "b0015b93-8a5d-461d-9991-3cfa23e3296f",
    },
    {
        "profile": "GoodTimeWithScar",
        "hermit_key": "cae9554c-31be-47e2-ba2b-4b8867adacc5",
    },
    {
        "profile": "Tinfoilchef",
        "hermit_key": "cbf33660-3994-42c3-8d2f-6a1a84d56dea",
    },
    {
        "profile": "Stressmonster101",
        "hermit_key": "cfaefb14-46d5-473b-9e8e-67ecbf119df7",
    },
    {
        "profile": "Keralis1",
        "hermit_key": "ed260cac-54e4-4ee5-b4de-d289f197fa45",
    },
    {
        "profile": "impulseSV",
        "hermit_key": "f6fe2200-609d-4fe6-88b6-529d59ee5b71",
    },
    {
        "profile": "Zedaph",
        "hermit_key": "f9c3c385-f403-403c-b5b7-867e012e9660",
    },
]

local_user = "culley"
# OSX path
path = Path(
    "/Users/{user}/Library/Application Support/minecraft/saves/hermitcraft9/".format(user=local_user)
)
stats_files = list(path.rglob("stats/*.json"))

def minecraft_key(key_name):
    """JSON nodes in minraft files are prefixed with minecraft: - this function removes the prefix so that the keys can be used in plot labels"""
    return "minecraft:{key}".format(key=key_name)


def merge_data_frames(df, df2, df_name):
    """Minecraft stats files contain separate datasets for: mined, crafted, broken, custom, dropped, killed, picked_up, and used. This function merges these datasets based on the keys, creating a column for each statistic"""
    return pd.merge(
        df,
        pd.DataFrame({"minecraft_key": df2.keys(), df_name: df2.values()}),
        how="left",
        left_on="minecraft_key",
        right_on="minecraft_key",
    )

def find_profile_by_filename(path, data_list):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in data_list:
        if dictionary.get('hermit_key') == filename:
            return dictionary.get('profile', 'none')
    
    # Return 'none' if no match found
    return 'none'
    
def world_stats(path):
    hermit = find_profile_by_filename(path, hermits)

    if hermit is None:
        return pd.DataFrame()

    df = pd.read_json(path)
    stats = df["stats"]
    
    broken = stats[minecraft_key("broken")] if minecraft_key("broken") in stats else {}
    crafted = (
        stats[minecraft_key("crafted")] if minecraft_key("crafted") in stats else {}
    )
    custom = stats[minecraft_key("custom")] if minecraft_key("custom") in stats else {}
    dropped = (
        stats[minecraft_key("dropped")] if minecraft_key("dropped") in stats else {}
    )
    killed = stats[minecraft_key("killed")] if minecraft_key("killed") in stats else {}
    mined = stats[minecraft_key("mined")] if minecraft_key("mined") in stats else {}
    picked_up = (
        stats[minecraft_key("picked_up")] if minecraft_key("picked_up") in stats else {}
    )
    used = stats[minecraft_key("used")] if minecraft_key("used") in stats else {}

    df = pd.DataFrame()

    # create unique list of keys from all dictionaries
    df["minecraft_key"] = list(
        set(broken.keys())
        | set(crafted.keys())
        | set(custom.keys())
        | set(dropped.keys())
        | set(killed.keys())
        | set(mined.keys())
        | set(picked_up.keys())
        | set(used.keys())
    )
    df = merge_data_frames(df, broken, "broken")
    df = merge_data_frames(df, crafted, "crafted")
    df = merge_data_frames(df, custom, "custom")
    df = merge_data_frames(df, dropped, "dropped")
    df = merge_data_frames(df, killed, "killed")
    df = merge_data_frames(df, mined, "mined")
    df = merge_data_frames(df, picked_up, "picked_up")
    df = merge_data_frames(df, used, "used")
    
    # remove minecraft: prefix
    df["minecraft_key"] = [
        re.sub(r"minecraft:", "", str(x)) for x in df["minecraft_key"]
    ]
    df["hermit"] = hermit
    df["wood_type"] = df["minecraft_key"].str.extract(
        r"(dark_oak|birch|oak|acacia|spruce|jungle|mangrove|cherry|bamboo)"
    )
    df = df.fillna(0)

    df = df.astype(
        {
            "broken": "int",
            "broken": "int",
            "crafted": "int",
            "custom": "int",
            "dropped": "int",
            "killed": "int",
            "mined": "int",
            "picked_up": "int",
            "used": "int",
        }
    )

    return df

minecraft_stats = pd.DataFrame()
parse_errors = pd.DataFrame(columns=["file_name"])
for file_name in stats_files:
   #try:
    minecraft_stats = pd.concat([minecraft_stats, world_stats(file_name)])
    #except:
    #    print(file_name)
    #    parse_errors.loc[len(parse_errors.index)] = file_name.name


# not currently shown on report:
parse_error_summary = parse_errors.groupby("file_name").count()

print(parse_error_summary)

#minecraft_stats = minecraft_stats.set_index(['minecraft_key', 'hermit'])
#if not minecraft_stats.index.is_unique:
#    raise ValueError("The multi-index is not unique.")
    
#minecraft_stats.to_json('hermitcraft9.json')
minecraft_stats.to_csv('hermitcraft9.csv')

#minecraft_stats.head()

#torches = minecraft_stats[minecraft_stats['minecraft_key'] == 'torch']

#torches.head()
#hermits


Empty DataFrame
Columns: []
Index: []


# 100 Top Mob Kill Stats for Hermitcraft Season 9


In [3]:
import re

def add_ordinal_suffix(num):
    # Check for 11-13 because those are the exceptions to the standard rules
    if 11 <= num % 100 <= 13:
        suffix = 'th'
    else:
        # Determine the suffix based on the last digit
        suffixes = {1: 'st', 2: 'nd', 3: 'rd'}
        suffix = suffixes.get(num % 10, 'th')

    return f"{num}{suffix}"
    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide):
    numbers = re.findall(r'\d+', next_slide)
    countdown = 100 - int(numbers[0]) + 2
    if countdown == 1:
        refresh = 10000
    elif countdown < 11:
        refresh = 6000
    elif countdown > 50:
        refresh = 2000
    else:
        refresh = 3000
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Mob Kill Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-5" style="padding-top:130px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">Top Mob Kill Stats</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card no-boarder" style="width: 400px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                                <center>
                                <img src="../{item_image}" style="max-width:128px; max-height:128px;" class="px-2"><br />
                                <span class="badge text-bg-primary">{item_name}</span>
                                </center>
                            </div>
                            <div class="col">
                            <center>
                                <img src="../{hermit_image}" width="128" height="128" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">{item_name}(s) killed by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh)


def generate_carousel_item_summary(hermit_name, hermit_image, total_kills, next_slide):
    # Define the HTML structure for the summary slide
    # Replace this with your actual HTML template
    # file:///Users/culley/Documents/clones/HermitcraftStats/slides/slide170.html
    return f"""
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Mob Kill Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, 5000); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>
                <div class="m-5" style="padding-top:130px;">
        <!-- <div><h3 class="text-center">Hermitcraft Season 9 Mob Kill Stats</h3></div> -->
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card" style="width: 400px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                                <img src="../{hermit_image}" width="128" height="128" class="px-2">
                                <span class="badge text-bg-success">{hermit_name}</span>
                            </div>
                        </div>
                    </div>
                    <div class="card-body">
                        <hr>
                        <h3 class="card-title"><span class="badge text-bg-danger">{hermit_name}'s total kills</span></h3>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{total_kills}</h1></span></p>
                    </div>
                </div>
            </div>
        </div>
        
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>
    </body>
    </html>
    """

file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

df = df[df["killed"] > 0]
df = df[df["hermit"] != 'none']

grouped_df = df.groupby(['hermit', 'minecraft_key'])['killed'].sum().reset_index()
# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1
show_summary_slides = False

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
sorted_df = grouped_df.sort_values(by=['killed'], ascending=[False])
sorted_df = sorted_df.head(100).sort_values(by=['killed'], ascending=[True])

#sorted_df = sorted_df[sorted_df["killed"] > 500]


def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None

# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():
    if current_hermit != row['hermit']:
        if current_hermit is not None and show_summary_slides is True:
            # Add a summary slide for the previous hermit
            hermit_key = find_hermit_key(current_hermit, hermits)
            summary_html = generate_carousel_item_summary(current_hermit, f"heads/{hermit_key}.png", "{:,}".format(total_kills), f"slide{slide_index + 1}.html")
            with open(f"slides/slide{slide_index}.html", "w") as file:
                file.write(summary_html)
            slide_index += 1
        current_hermit = row['hermit']
        total_kills = 0

    total_kills += row['killed']
    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_key = find_hermit_key(row['hermit'], hermits)
    item_html = generate_carousel_item(
        f"heads/{hermit_key}.png",
        f"mobs/{row['minecraft_key']}.webp",
        row['hermit'],
        row['minecraft_key'].replace("_", " ").title(),
        "{:,}".format(row['killed']),
        next_slide
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1
    
# Add a summary slide for the last hermit
if current_hermit is not None and show_summary_slides is True:
    hermit_key = find_hermit_key(current_hermit, hermits)
    summary_html = generate_carousel_item_summary(current_hermit, f"heads/{hermit_key}.png", total_kills, "slide1.html")
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(summary_html)


    
#print(df["hermit"].unique())


# 100 Top Blocks Mined in Hermitcraft Season 9


In [13]:
import re

def add_ordinal_suffix(num):
    # Check for 11-13 because those are the exceptions to the standard rules
    if 11 <= num % 100 <= 13:
        suffix = 'th'
    else:
        # Determine the suffix based on the last digit
        suffixes = {1: 'st', 2: 'nd', 3: 'rd'}
        suffix = suffixes.get(num % 10, 'th')

    return f"{num}{suffix}"
    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide):
    numbers = re.findall(r'\d+', next_slide)
    countdown = 100 - int(numbers[0]) + 2
    if countdown == 1:
        refresh = 10000
    elif countdown < 11:
        refresh = 6000
    elif countdown > 50:
        refresh = 2000
    else:
        refresh = 3000
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Mob Kill Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-5" style="padding-top:130px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">Top Blocks Mined</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card no-boarder" style="width: 400px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                                <center>
                                <img src="../{item_image}" style="width:128px; height:128px;" class="px-2"><br />
                                <span class="badge text-bg-primary">{item_name}</span>
                                </center>
                            </div>
                            <div class="col">
                            <center>
                                <img src="../{hermit_image}" width="128" height="128" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">{item_name} mined by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh)

file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

df = df[df["mined"] > 0]
df = df[df["hermit"] != 'none']

grouped_df = df.groupby(['hermit', 'minecraft_key'])['mined'].sum().reset_index()
# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
sorted_df = grouped_df.sort_values(by=['mined'], ascending=[False])
sorted_df = sorted_df.head(100).sort_values(by=['mined'], ascending=[True])

#sorted_df = sorted_df[sorted_df["killed"] > 500]


def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None

# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():

    total_kills += row['mined']
    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_key = find_hermit_key(row['hermit'], hermits)
    item_html = generate_carousel_item(
        f"heads/{hermit_key}.png",
        f"blocks/{row['minecraft_key']}.png",
        row['hermit'],
        row['minecraft_key'].replace("_", " ").title(),
        "{:,}".format(row['mined']),
        next_slide
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1
    
#print(df["hermit"].unique())


# Top 100 Blocks Crafted in Hermitcraft Season 9


In [16]:
import re

def add_ordinal_suffix(num):
    # Check for 11-13 because those are the exceptions to the standard rules
    if 11 <= num % 100 <= 13:
        suffix = 'th'
    else:
        # Determine the suffix based on the last digit
        suffixes = {1: 'st', 2: 'nd', 3: 'rd'}
        suffix = suffixes.get(num % 10, 'th')

    return f"{num}{suffix}"
    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide):
    numbers = re.findall(r'\d+', next_slide)
    countdown = 100 - int(numbers[0]) + 2
    if countdown == 1:
        refresh = 6000
    elif countdown < 11:
        refresh = 3000
    elif countdown > 50:
        refresh = 1000
    else:
        refresh = 2000
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Mob Kill Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-5" style="padding-top:130px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">Top Crafts</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card no-boarder" style="width: 400px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                                <center>
                                <img src="../{item_image}" style="width:128px; height:128px;" class="px-2"><br />
                                <span class="badge text-bg-primary">{item_name}</span>
                                </center>
                            </div>
                            <div class="col">
                            <center>
                                <img src="../{hermit_image}" width="128" height="128" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">{item_name} crafted by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh)

file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

df = df[df["crafted"] > 0]
df = df[df["hermit"] != 'none']

grouped_df = df.groupby(['hermit', 'minecraft_key'])['crafted'].sum().reset_index()
# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
sorted_df = grouped_df.sort_values(by=['crafted'], ascending=[False])
sorted_df = sorted_df.head(100).sort_values(by=['crafted'], ascending=[True])

#sorted_df = sorted_df[sorted_df["killed"] > 500]


def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None

# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():

    total_kills += row['crafted']
    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_key = find_hermit_key(row['hermit'], hermits)
    item_html = generate_carousel_item(
        f"heads/{hermit_key}.png",
        f"blocks/{row['minecraft_key']}.png",
        row['hermit'],
        row['minecraft_key'].replace("_", " ").title(),
        "{:,}".format(row['crafted']),
        next_slide
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1
    
#print(df["hermit"].unique())


# Dimond Ore Mined in Hermitcraft Season 9


In [22]:
import re

    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide):
    numbers = re.findall(r'\d+', next_slide)
    countdown = 26 - int(numbers[0]) + 2
    if countdown == 1:
        refresh = 6000
    elif countdown < 5:
        refresh = 4000
    else:
        refresh = 3000
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Mob Kill Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-5" style="padding-top:130px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">Diamond Ore Mined</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card no-boarder" style="width: 450px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                            <center>
                                <img src="../{hermit_image}" width="128" height="128" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                            <div class="col">
                                <center>
                                <img src="../{item_image}" style="width:128px; height:128px;" class="px-2"><br />
                                <span class="badge text-bg-primary">{item_name}</span>
                                </center>
                            </div>

                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">{item_name}</h5>
                         <h5 class="card-title">mined by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh)

file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

df = df[df["mined"] > 0]
df = df[df["hermit"] != 'none']
df = df[df["minecraft_key"] == 'deepslate_diamond_ore']

grouped_df = df.groupby(['hermit', 'minecraft_key'])['mined'].sum().reset_index()
# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
sorted_df = grouped_df.sort_values(by=['mined'], ascending=[False])
sorted_df = sorted_df.head(100).sort_values(by=['mined'], ascending=[True])

#sorted_df = sorted_df[sorted_df["killed"] > 500]


def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None

# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():

    total_kills += row['mined']
    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_key = find_hermit_key(row['hermit'], hermits)
    item_html = generate_carousel_item(
        f"heads/{hermit_key}.png",
        f"blocks/{row['minecraft_key']}.png",
        row['hermit'],
        row['minecraft_key'].replace("_", " ").title(),
        "{:,}".format(row['mined']),
        next_slide
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1
    
#print(df["hermit"].unique())


# Total Players Killed in Hermitcraft Season 9


In [39]:
import re

    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide):
    numbers = re.findall(r'\d+', next_slide)
    countdown = 26 - int(numbers[0])
    if countdown == 1:
        refresh = 6000
    elif countdown < 5:
        refresh = 4000
    else:
        refresh = 3000
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-5" style="padding-top:130px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">Top PVP Kills</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card no-boarder" style="width: 450px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                            <center>
                                <img src="../{hermit_image}" width="128" height="128" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                            <div class="col">
                                <center>
                                <img src="../{item_image}" style="max-width:128px; max-height:128px;" class="px-2"><br />
                                <span class="badge text-bg-primary">Other Hermit</span>
                                </center>
                            </div>

                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">Other Hermits</h5>
                         <h5 class="card-title">killed by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh)

file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

df = df[df["killed"] > 0]
df = df[df["hermit"] != 'none']
df = df[df["minecraft_key"] == 'player']

grouped_df = df.groupby(['hermit', 'minecraft_key'])['killed'].sum().reset_index()
# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
sorted_df = grouped_df.sort_values(by=['killed'], ascending=[False])
sorted_df = sorted_df.head(100).sort_values(by=['killed'], ascending=[True])

#sorted_df = sorted_df[sorted_df["killed"] > 500]


def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None

# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():

    total_kills += row['killed']
    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_key = find_hermit_key(row['hermit'], hermits)
    item_html = generate_carousel_item(
        f"heads/{hermit_key}.png",
        f"mobs/{row['minecraft_key']}.webp",
        row['hermit'],
        row['minecraft_key'].replace("_", " ").title(),
        "{:,}".format(row['killed']),
        next_slide
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1
    


# Lecterns Broken

In [157]:
import re

    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide):
    numbers = re.findall(r'\d+', next_slide)
    countdown = 23 - int(numbers[0]) +2

    refresh = 2300
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-5" style="padding-top:130px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">Lecterns Mined</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card no-boarder" style="width: 450px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                            <center>
                                <img src="../{hermit_image}" width="128" height="128" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                            <div class="col">
                                <center>
                                <img src="../{item_image}" height="128" width="128" class="px-2"><br />
                                <span class="badge text-bg-primary">Lecterns</span>
                                </center>
                            </div>

                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">Lecterns</h5>
                         <h5 class="card-title">mined by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh)

file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)


df = df[df["mined"] > 0]
df = df[df["hermit"] != 'none']
df = df[df["minecraft_key"] == 'lectern']

grouped_df = df.groupby(['hermit'])['mined'].max().reset_index()

#grouped_df.head()


# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
sorted_df = grouped_df.sort_values(by=['mined'], ascending=[True])
#sorted_df = sorted_df.head(100).sort_values(by=['broken'], ascending=[True])

#sorted_df = sorted_df[sorted_df["killed"] > 500]


def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None

# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():

    total_kills += row['mined']
    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_key = find_hermit_key(row['hermit'], hermits)
    item_html = generate_carousel_item(
        f"heads/{hermit_key}.png",
        f"blocks/lectern.png",
        row['hermit'],
        'Lectern',
        "{:,}".format(row['mined']),
        next_slide
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1
    



In [27]:
flowers = [
    "poppy", "dandelion", "blue_orchid", "allium", "azure_bluet", 
    "red_tulip", "orange_tulip", "white_tulip", "pink_tulip", 
    "oxeye_daisy", "cornflower", "lily_of_the_valley", "wither_rose", 
    "sunflower", "lilac", "rose_bush", "peony"
]


# Top flower gardeners in hermitcraft season 9


In [43]:
import re
import random

    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide):
    numbers = re.findall(r'\d+', next_slide)
    countdown = 26 - int(numbers[0]) + 1
    random_flower = random.choice(flowers)
    if countdown == 1:
        refresh = 6000
    elif countdown < 5:
        refresh = 4000
    else:
        refresh = 3000
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-5" style="padding-top:130px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">Top Hermits Planting Flowers</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card no-boarder" style="width: 450px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                            <center>
                                <img src="../{hermit_image}" width="128" height="128" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                            <div class="col">
                                <center>
                                <img src="../blocks/{item_image}.png" style="width:128px; height:128px;" class="px-2"><br />
                                <span class="badge text-bg-primary">Total Flowers Planted</span>
                                </center>
                            </div>

                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">Total flowers</h5>
                         <h5 class="card-title">planted by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=random_flower, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh)

file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

df = df[df["used"] > 0]
df = df[df["hermit"] != 'none']
df = df[df["minecraft_key"].isin(flowers)]

grouped_df = df.groupby(['hermit'])['used'].sum().reset_index()
# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
sorted_df = grouped_df.sort_values(by=['used'], ascending=[False])
sorted_df = sorted_df.head(100).sort_values(by=['used'], ascending=[True])

#sorted_df = sorted_df[sorted_df["killed"] > 500]


def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None

# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():

    total_kills += row['used']
    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_key = find_hermit_key(row['hermit'], hermits)
    item_html = generate_carousel_item(
        f"heads/{hermit_key}.png",
        f"blocks/nada.png",
        row['hermit'],
        'flowers',
        "{:,}".format(row['used']),
        next_slide
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1
    


# Individual Hermit's top 100 stats


In [119]:
import pandas as pd

import re
import random

    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide, stat_type):
    if not os.path.exists(item_image):
        # Get the filename without the extension
        filename_without_ext, _ = os.path.splitext(os.path.basename(item_image))

        # Construct the alternative path in the mobs directory with .webp extension
        alternative_image = os.path.join('mobs', filename_without_ext + '.webp')

        # Check if the alternative path exists
        if not os.path.exists(alternative_image):
            # Handle the case where the image is not found in either directory
            item_image = 'path_to_default_image.png'  # Replace with your default image path
        else:
            item_image = alternative_image
    numbers = re.findall(r'\d+', next_slide)
    countdown = 100 - int(numbers[0]) + 2

    if countdown == 1:
        refresh = 5000
    elif countdown < 5:
        refresh = 2000
    else:
        refresh = 1000
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-4" style="padding-top:90px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">{hermit_name}<br />Top 100  Statistics</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 450px;">
                <div class="card no-boarder" style="width: 450px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                            <center>
                                <img src="{hermit_image}" style="max-width:128px; max-height:128px;" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                            <div class="col">
                                <center>
                                <img src="../{item_image}" height="96px" width="96px" class="px-2"><br />
                                <span class="badge text-bg-primary">{item_name}</span>
                                </center>
                            </div>

                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">Total {item_name}</h5>
                         <h5 class="card-title"><span class="badge text-bg-info">{stat_type}</span> by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh, stat_type=stat_type)


slide_index = 1
hermit_name = 'Mumbo'
file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

# Filter the DataFrame
df = df[df["hermit"] == hermit_name]

# Columns to be transformed
value_vars = ['broken', 'crafted', 'dropped', 'killed', 'mined', 'picked_up', 'used']

# Use melt to transform the DataFrame
flattened_df = df.melt(id_vars=['minecraft_key'], 
                       value_vars=value_vars, 
                       var_name='stat_type',
                       value_name='total_amount')

# Sort the DataFrame by total_amount in descending order
sorted_df = flattened_df.sort_values(by='total_amount', ascending=False)
sorted_df = sorted_df.head(100).sort_values(by='total_amount', ascending=True)


def find_hermit_skin(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('skin', None)
    
    # Return 'none' if no match found
    return None

def replace_middle_space_with_br(string):
    # Find all the positions of spaces in the string
    space_positions = [pos for pos, char in enumerate(string) if char == " "]

    # Check if there are exactly three spaces
    if len(space_positions) == 3:
        # Find the position of the middle space
        middle_space_position = space_positions[1]

        # Replace the middle space with a <br> tag
        string = string[:middle_space_position] + "<br>" + string[middle_space_position + 1:]

    return string
    
# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():

    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_skin = f"https://minepic.org/skin/{hermit_name}"
    
    minecraft_key_string = row['minecraft_key'].replace("_", " ").title()
    formatted_mc_key = replace_middle_space_with_br(minecraft_key_string)
    item_html = generate_carousel_item(
        f"{hermit_skin}",
        f"blocks/{row['minecraft_key']}.png",
        hermit_name.title(),
        formatted_mc_key,
        "{:,}".format(row['total_amount']),
        next_slide,
        row['stat_type'].replace("_", " ").title(),
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1





# Create wood type bar plots for each hermit


In [188]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Load data
file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

# Filter out rows where wood_type is '0'
df = df[df["wood_type"] != '0']
df = df[df["hermit"] != 'none']

# Replace 'dark_oak' with 'dark oak' and convert to lower case
df['wood_type'] = df['wood_type'].str.replace('_', ' ').str.lower()

# Summing numeric columns and grouping
numeric_cols = ['broken', 'crafted', 'dropped', 'killed', 'mined', 'picked_up', 'used']
df['total'] = df[numeric_cols].sum(axis=1)
grouped_df = df.groupby(['hermit', 'wood_type'])['total'].sum().reset_index()

# Create the 'wood' subdirectory if it doesn't exist
os.makedirs('wood', exist_ok=True)

# Iterate over each hermit and create a separate plot
for hermit in grouped_df['hermit'].unique():
    # Filter data for the current hermit
    hermit_df = grouped_df[grouped_df['hermit'] == hermit]

    # Sort the data by 'total' column in descending order
    hermit_df = hermit_df.sort_values(by='total', ascending=False)

    # Create the square bar plot
    plt.figure(figsize=(6, 6))
    sns.barplot(data=hermit_df, x='wood_type', y='total', palette=color_map)
    plt.xticks(rotation=45)
    plt.xlabel('Wood Type')
    plt.ylabel('')
    plt.ylabel('')

    # Remove chart title
    plt.title('')

    # Save the plot
    plt.savefig(f'wood/{hermit}.png', bbox_inches='tight', pad_inches=0)

    # Close the plot to free memory
    plt.close()


In [70]:
import pandas as pd

import re
import random

    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide):
    if not os.path.exists(item_image):
        # Get the filename without the extension
        filename_without_ext, _ = os.path.splitext(os.path.basename(item_image))

        # Construct the alternative path in the mobs directory with .webp extension
        alternative_image = os.path.join('mobs', filename_without_ext + '.webp')

        # Check if the alternative path exists
        if not os.path.exists(alternative_image):
            # Handle the case where the image is not found in either directory
            item_image = 'path_to_default_image.png'  # Replace with your default image path
        else:
            item_image = alternative_image
    numbers = re.findall(r'\d+', next_slide)
    countdown = 26 - int(numbers[0])

    refresh = 2000
    
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-4" style="padding-top:60px;">
            <div style="height: 150px;">
                <h6 class="text-center"><small>Hermitcraft Season 9</small></h6>
                <h5 class="text-center">{hermit_name}<br />preferred {item_name}!</h5>
            </div>
            <div class="d-flex justify-content-center align-items-center" style="height: 450px;">
                <div class="card no-boarder" style="width: 450px; border: none; box-shadow: none;">
                    <div class="container">
                        <div class="row">
                            <div class="col">
                            <center>
                                <img src="{hermit_image}" style="max-width:128px; max-height:128px;" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                            <div class="col">
                                <center>
                                <img src="../{item_image}" height="96px" width="96px" class="px-2"><br />
                                <span class="badge text-bg-primary">{item_name}</span>
                                </center>
                            </div>

                        </div>
                    </div>
                    <div class="card-body"><center>
                        <img src="../wood/{hermit_name}.png" width="350" height="350" class="px-2">
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh)




# Load data
file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

# Filter out rows where wood_type is '0'
df = df[df["wood_type"] != '0']
df = df[df["hermit"] != 'none']

# Replace 'dark_oak' with 'dark oak' and convert to lower case
df['wood_type'] = df['wood_type'].str.replace('_', ' ').str.lower()

# Summing numeric columns and grouping
numeric_cols = ['broken', 'crafted', 'dropped', 'killed', 'mined', 'picked_up', 'used']
df['total'] = df[numeric_cols].sum(axis=1)
grouped_df = df.groupby(['hermit', 'wood_type'])['total'].sum().reset_index()

# Find the wood type with the maximum total for each hermit
max_wood_type = grouped_df.groupby('hermit')['total'].idxmax()
result_df = grouped_df.loc[max_wood_type]

result_df = result_df.sort_values(by='total', ascending=False)


def find_hermit_skin(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('skin', None)
    
    # Return 'none' if no match found
    return None

def replace_middle_space_with_br(string):
    # Find all the positions of spaces in the string
    space_positions = [pos for pos, char in enumerate(string) if char == " "]

    # Check if there are exactly three spaces
    if len(space_positions) == 3:
        # Find the position of the middle space
        middle_space_position = space_positions[1]

        # Replace the middle space with a <br> tag
        string = string[:middle_space_position] + "<br>" + string[middle_space_position + 1:]

    return string
    
slide_index = 1
os.makedirs('slides', exist_ok=True)

for index, row in result_df.iterrows():

    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_skin = f"https://minepic.org/skin/{row['hermit']}"
    
    wood_type = row['wood_type']
    wood_type = wood_type.replace(' ', '_').lower()
    
    item_html = generate_carousel_item(
        f"{hermit_skin}",
        f"blocks/{wood_type}_log.png",
        row['hermit'].title(),
        row['wood_type'],
        "{:,}".format(row['total']),
        next_slide,
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1





In [42]:
import pandas as pd


# Load data
file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)

# Filter out rows where wood_type is '0'
df = df[df["wood_type"] != '0']
df = df[df["hermit"] != 'none']

# Replace 'dark_oak' with 'dark oak' and convert to lower case
df['wood_type'] = df['wood_type'].str.replace('_', ' ').str.lower()

# Summing numeric columns and grouping
numeric_cols = ['broken', 'crafted', 'dropped', 'killed', 'mined', 'picked_up', 'used']
df['total'] = df[numeric_cols].sum(axis=1)
grouped_df = df.groupby(['hermit', 'wood_type'])['total'].sum().reset_index()

# Find the wood type with the maximum total for each hermit
max_wood_type = grouped_df.groupby('hermit')['total'].idxmax()
result_df = grouped_df.loc[max_wood_type]

result_df = result_df.sort_values(by='total', ascending=False)



print(result_df)



               hermit wood_type   total
205      joehillssays     birch  496856
16             Docm77       oak  307890
155     falsesymmetry  dark oak  245031
43   GoodTimeWithScar       oak  239386
53           Keralis1    spruce  228872
96              Tango    spruce  209400
75          Renthedog  dark oak  174309
175         hypnotizd    spruce  160514
70    PearlescentMoon    spruce  157962
184            iJevin    spruce  136285
122            Xisuma       oak  122693
35          GeminiTay    spruce  105689
84   Stressmonster101  dark oak   95550
141        ZombieCleo    spruce   88121
219         xBCrafted    spruce   84305
146         cubfan135  dark oak   79818
25               Etho       oak   74517
102       VintageBeef     birch   74491
186         impulseSV    bamboo   72128
162             grian     birch   62311
8         BdoubleO100    spruce   46906
198          iskall85  dark oak   43222
131            Zedaph       oak   20090
55              Mumbo    bamboo   14867


In [113]:
import re
import csv
from pathlib import Path

def extract_and_append_to_csv(input_filename, hermit_name, output_filename):
    # Regular expression pattern to match dates in quotes
    date_pattern = r'"(\d{4}-\d{2}-\d{2}) \d{2}:\d{2}:\d{2} \+\d{4}"'

    # Initialize a list to store extracted data
    data = []

    # Open the input file for reading
    with open(input_filename, 'r') as file:
        # Read the file line by line
        for line in file:
            # Find all date matches in the line
            dates = re.findall(date_pattern, line)
            
            # If dates are found, append them to the data list with the hermit name
            if dates:
                for date in dates:
                    data.append([date, hermit_name, 1])  # Changed order to date, hermit_name

    # Append the data to an existing CSV file
    with open(output_filename, 'a', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        # Don't write the header if the file already exists
        if csv_file.tell() == 0:
            csv_writer.writerow(['Date', 'Name', 'Amount'])  # Updated header
        csv_writer.writerows(data)

# Example usage:
# extract_and_append_to_csv('input.txt', 'HermitX', 'achievements.csv')

local_user = "culley"
# OSX path
path = Path("/Users/{user}/Library/Application Support/minecraft/saves/hermitcraft9/advancements/".format(user=local_user))
stats_files = list(path.rglob("*.json"))
output_csv = 'achievements.csv'

for file_path in stats_files:
    hermit = find_profile_by_filename(file_path, hermits)
    if hermit != 'none':
        extract_and_append_to_csv(file_path, hermit, output_csv)


# Create Bar Chart Race

In [114]:
import os
import pandas as pd
import bar_chart_race as bcr

# Function to delete the file if it exists
def delete_file_if_exists(file_path):
    if os.path.exists(file_path):
        os.remove(file_path)

# Function to prepare data for bar chart race
def prepare_data_for_bar_chart_race(df):
    # Convert 'Advancement Date' to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Set date as index and count cumulative advancements for each hermit
    df['Amount'] = 1
    df_cumulative = df.groupby(['Name', pd.Grouper(key='Date', freq='D')]).count().groupby(level=0).cumsum()

    # Reshape the data for the bar chart race
    df_pivot = df_cumulative.reset_index().pivot(index='Date', columns='Name', values='Amount').fillna(method='ffill').fillna(0)

    return df_pivot

# Function to create a bar chart race
def create_bar_chart_race(df_pivot, filename):
    # Create and save the bar chart race
    bcr.bar_chart_race(
        df=df_pivot,
        filename=filename,
        n_bars=26,
        period_fmt='%B %d, %Y',
        steps_per_period=10,
        title='Total Hermit Advancements Season 9',
        #end_period_pause=10,
        label_bars=True,
        bar_size=.95,
        title_size='',
        bar_label_size=7,
        tick_label_size=7,
        shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
        scale='linear',
        writer=None,
        fig=None,
        bar_kwargs={'alpha': .7},
        filter_column_colors=False
    )

# CSV file path
csv_file_path = 'achievements.csv'

# Insert your code here to generate and append data to 'achievements.csv'
# ...

# Check if the file exists before attempting to read it
if os.path.exists(csv_file_path):
    # Load your CSV data
    data = pd.read_csv(csv_file_path)

    # Prepare the data
    df_pivot = prepare_data_for_bar_chart_race(data)

    # File path for the output video
    output_file_path = 'achievements.mp4'

    extra_periods = 30  # Adjust this based on your desired duration

    # Duplicate the final row of the DataFrame
    last_row = df_pivot.iloc[[-1] * extra_periods]
    
    # Append these rows to the original DataFrame
    df_pivot_extended = pd.concat([df_pivot, last_row])

    # Create the bar chart race
    create_bar_chart_race(df_pivot_extended, output_file_path)
else:
    print("No data available to create bar chart race.")


/var/folders/dv/ts7drh5n0hb48yx2_5vpm3kw0000gq/T/ipykernel_15879/4010500348.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_pivot = df_cumulative.reset_index().pivot(index='Date', columns='Name', values='Amount').fillna(method='ffill').fillna(0)
/opt/homebrew/Caskroom/miniconda/base/envs/dashboards/lib/python3.12/site-packages/bar_chart_race/_make_chart.py:889: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_values.iloc[:, 0] = df_values.iloc[:, 0].fillna(method='ffill')
/opt/homebrew/Caskroom/miniconda/base/envs/dashboards/lib/python3.12/site-packages/bar_chart_race/_make_chart.py:286: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_yticklabels(self.df_values.columns)
/opt/homebrew/Caskroom/miniconda/base/envs/

# crafting tables crafted

In [11]:
import re
import glob

    
def generate_carousel_item(hermit_image, item_image, hermit_name, item_name, amount, next_slide, column, row_count):
    numbers = re.findall(r'\d+', next_slide)
    countdown = row_count - int(numbers[0]) + 2

    refresh = 2300
    html_template = """
    <!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Hermitcraft Season 9 Stats</title>
    <!-- Bootstrap CSS -->
    <link rel="stylesheet" href="../bootstrap/css/bootstrap.min.css">
        <script type="text/javascript">
        setTimeout(function() {{
            window.location.href = '{next_slide}';
        }}, {refresh}); // 3000 milliseconds = 3 seconds
    </script>

</head>
<body>

        <div class="m-5" style="padding-top:130px;">
            <div>
                <h5 class="text-center">Hermitcraft Season 9</h5>
                <h1 class="text-center"><span class="badge text-bg-danger">{countdown}</span></h1>
                <h3 class="text-center">{item_name} {column}</h3>
            </div>
            <div class="d-flex justify-content-center align-items-center m-5" style="height: 400px;">
                <div class="card no-boarder" style="width: 450px; border: none; box-shadow: none;">
                    <div class="container p-5">
                        <div class="row">
                            <div class="col">
                            <center>
                                <img src="../{hermit_image}" width="128" height="128" class="px-2"><br />
                                <span class="badge text-bg-success">{hermit_name}</span>
                                </center>
                            </div>
                            <div class="col">
                                <center>
                                <img src="../{item_image}" height="128" width="128" class="px-2"><br />
                                <span class="badge text-bg-primary">{item_name}</span>
                                </center>
                            </div>

                        </div>
                    </div>
                    <div class="card-body"><center>
                        <h5 class="card-title">{item_name}</h5>
                         <h5 class="card-title">{column} by {hermit_name}</h5>
                        <p class="card-text"><span class="badge text-bg-primary"><h1 class="display-1">{amount}</h1></span></p>
                    </center>
                    </div>
                </div>
            </div>
        </div>
        <!-- Bootstrap JS -->
<script src="../bootstrap/js/bootstrap.bundle.min.js"></script>

</body>
</html>
    """
    return html_template.format(hermit_image=hermit_image, item_image=item_image, hermit_name=hermit_name, item_name=item_name, amount=amount, next_slide=next_slide, countdown=countdown, refresh=refresh, column=column)


slides_dir = 'slides'

# Create a pattern to match all HTML files
pattern = os.path.join(slides_dir, '*.html')

# Find all HTML files in the directory
html_files = glob.glob(pattern)

# Loop through the found HTML files and delete each one
for file_path in html_files:
    try:
        os.remove(file_path)
    except Exception as e:
        print(f"Error deleting {file_path}: {e}")
        
file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)


column = 'used'
mc_key = 'firework_rocket'
df = df[df[column] > 0]
df = df[df["hermit"] != 'none']
df = df[df["minecraft_key"] == mc_key]



grouped_df = df.groupby(['hermit'])[column].max().reset_index()

row_count = len(grouped_df)

#grouped_df.head()


# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
sorted_df = grouped_df.sort_values(by=[column], ascending=[True])
#sorted_df = sorted_df.head(100).sort_values(by=['broken'], ascending=[True])

#sorted_df = sorted_df[sorted_df["killed"] > 500]


def find_hermit_key(hermit, hermits):
    # Extract the base filename without extension
    filename = os.path.splitext(os.path.basename(path))[0]
    
    # Search through the list of dictionaries
    for dictionary in hermits:
        if dictionary.get('profile') == hermit:
            return dictionary.get('hermit_key', None)
    
    # Return 'none' if no match found
    return None

# Ensure the 'slides' subdirectory exists
os.makedirs('slides', exist_ok=True)

for index, row in sorted_df.iterrows():

    total_kills += row[column]
    next_slide = f"slide{slide_index + 1}.html" #if index < len(sorted_df) - 1 else "slide1.html"
    hermit_key = find_hermit_key(row['hermit'], hermits)
    item_html = generate_carousel_item(
        f"heads/{hermit_key}.png",
        f"blocks/{mc_key}.png",
        row['hermit'],
        mc_key.replace('_', ' ').title(),
        "{:,}".format(row[column]),
        next_slide,
        column.title(),
        row_count
    )
    with open(f"slides/slide{slide_index}.html", "w") as file:
        file.write(item_html)
    slide_index += 1
    



In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# Load data
file_path = 'hermitcraft9.csv'
df = pd.read_csv(file_path)


food = [
    "apple", "baked_potato", "beetroot", "beetroot_soup", "bread", "cake", "carrot", 
    "chorus_fruit", "cooked_chicken", "cooked_cod", "cooked_mutton", "cooked_porkchop", 
    "cooked_rabbit", "cooked_salmon", "cookie", "dried_kelp", "enchanted_golden_apple", 
    "golden_apple", "golden_carrot", "honey_bottle", "melon_slice", "mushroom_stew", 
    "poisonous_potato", "potato", "pufferfish", "pumpkin_pie", "rabbit_stew", "raw_beef", 
    "raw_chicken", "raw_cod", "raw_mutton", "raw_porkchop", "raw_rabbit", "raw_salmon", 
    "rotten_flesh", "spider_eye", "steak", "suspicious_stew", "sweet_berries", "tropical_fish"
]


df = df[df["minecraft_key"].isin(food)]
df = df[df["hermit"] != 'none']

grouped_df = df.groupby(['hermit', 'minecraft_key'])['used'].sum().reset_index()

row_count = len(grouped_df)

#grouped_df.head()


# Track the current hermit and their total kills
current_hermit = None
total_kills = 0
slide_index = 1

# Hermit summary data
#sorted_df = grouped_df.sort_values(by=['hermit', 'minecraft_key'])
# top killed data
grouped_df = grouped_df.sort_values(by='used', ascending=[True])

grouped_df.head()

# Iterate over each hermit and create a separate plot
for hermit in grouped_df['hermit'].unique():
    # Filter data for the current hermit
    hermit_df = grouped_df[grouped_df['hermit'] == hermit]

    # Sort the data by 'total' column in descending order
    hermit_df = hermit_df.sort_values(by='used', ascending=False)

    # Create the square bar plot
    plt.figure(figsize=(6, 6))
    sns.barplot(data=hermit_df, x='minecraft_key', y='used')
    plt.xticks(rotation=45)
    plt.xlabel('All Food Eaten')
    plt.ylabel('')
    plt.ylabel('')

    # Remove chart title
    plt.title('')

    # Save the plot
    plt.savefig(f'wood/{hermit}.png', bbox_inches='tight', pad_inches=0)

    # Close the plot to free memory
    plt.close()


,hermit,minecraft_key,used
639,xBCrafted,tropical_fish,0
136,Keralis1,enchanted_golden_apple,0
375,ZombieCleo,beetroot,0
372,Zedaph,tropical_fish,0
140,Keralis1,melon_slice,0
